# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [14]:
%reload_ext autoreload
%autoreload 2

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json

from llmchem.utils import mk_dir,clean_vram

In [3]:
#dataset settings
n_test=5 #number of testing data
n_train_check=5 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=30 # number of reasoning data made by GPT

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_30k.csv"

#project path
project_dir="results/projects/240117test"

In [5]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")

In [7]:
#init model
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Prepare dataset

In [11]:

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
train_check_dataset=base_train_dataset[-n_train_check:]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [12]:
#prepare train dataset

# reason data made by GPT4
train_dataset=copy.deepcopy(example_reasoning_dataset)

# reason data made by model itself
for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
    with open(path) as f:
        train_dataset.append(json.load(f))

random.shuffle(train_dataset)
len(train_dataset)

30

In [13]:
#train model
clean_vram()
model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
train_result=train_model(model,tokenizer,train_dataset,
                epochs=epochs,
                lr=lr,
                per_device_train_batch_size=per_device_train_batch_size,
                gradient_checkpointing=gradient_checkpointing,
                )

Using fp16 mode


Map: 100%|██████████| 3000/3000 [00:00<00:00, 11267.51 examples/s]
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                             f"{project_dir}/eval",
                             )

In [ ]:
self_reasoning(model,tokenizer,example_reasoning_dataset,project_dir)